In [ ]:
# from google.colab import drive
# drive.mount("/content/drive")

In [ ]:
# load data
import pandas as pd
df = pd.read_csv("./traindata1.1.csv",engine="python")
df.drop(axis=1, inplace=True, columns=['UserID','Date/Time'] )
df.drop_duplicates(inplace=True)
# feature column and target
# ENGoogleTranslate | finalLabel5Classes | SN(Original Shona Tweet)
_feature_col = "SN(Original Shona Tweet)"
_target_col = "finalLabel5Classes"
df[[_feature_col,_target_col ]].head(5)
# df.columns

In [ ]:
# # text processing function 
import re
import string
def clean_text(text):
    # to lower case
    text = str(text)
    text = text.lower()
    # remove links
    text = re.sub('https:\/\/\S+', '', text) 
    # remove punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) 
    # remove next line     
    text = re.sub(r'[^ \w\.]', '', text) 
    # remove words containing numbers
    text = re.sub('\w*\d\w*', '', text)

    text = ' '.join( text.split() )
    
    return text# Create a new column called "Text" for collecting clean text
df['Text'] = df[_feature_col].apply(lambda x: clean_text(x))
df[['Text',_target_col]]

In [ ]:
# make label from target text data.
from sklearn.preprocessing import LabelEncoder

leObj = LabelEncoder()
leObj.fit(df[_target_col])
target_num = leObj.transform(df[_target_col])
target_num

In [ ]:
# seperate the data
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df['Text'], target_num, random_state=234, test_size=.2 )

In [ ]:
# install lazy predict model.
!pip install lazypredict

In [ ]:
# build text transformers using counvectorizers and tfidfTransformer.
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
# 
# pipe = Pipeline(steps=[
#     ('vect', CountVectorizer() ),
#     ('tfidf', TfidfTransformer() ),
# ])
# 
vect = CountVectorizer()
tfidf = TfidfTransformer()
# x_train.values.tolist()
# transform X_train data
td = vect.fit_transform(x_train.values.tolist())
X_train = tfidf.fit_transform(td)

# transform X_test data
td1 = vect.fit_transform(x_test.values.tolist())
X_test = tfidf.fit_transform(td1)

In [37]:
from sklearn.ensemble import GradientBoostingClassifier

In [46]:
from lazypredict.Supervised import LazyClassifier

# initialise lazy classifier object
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)

(<class 'ModuleNotFoundError'>, ModuleNotFoundError("No module named 'catboost'"), <traceback object at 0x13eac5580>)
(<class 'ModuleNotFoundError'>, ModuleNotFoundError("No module named 'lightgbm'"), <traceback object at 0x1753c1b00>)


NameError: name 'lightgbm' is not defined

In [ ]:
# fit data, on all classification models,
import numpy as np

# scipy.sparse.csr.csr_matrix
# X_train.toarray()
models, predictions = clf.fit( X_train.toarray(), X_test.toarray(), y_train, y_test )

In [ ]:
# Calculate performance of all models on test dataset
models

In [ ]:
model_dictionary = clf.provide_models(X_train,X_test,y_train,y_test)